# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [18]:
## Connect to a Cassandra instance on the local machine (localhost)

from cassandra.cluster import Cluster
cluster = Cluster(['localhost'])

session = cluster.connect()

#### Create Keyspace

In [19]:
## Create a Keyspace

session.execute('''CREATE KEYSPACE IF NOT EXISTS udacity \
                    WITH REPLICATION = { 'class': 'SimpleStrategy', \
                                        'replication_factor': 1}
                 ''')

#### Set Keyspace

In [20]:
## Set KEYSPACE to the keyspace specified above

session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



# Main Code:

## Query 1: Return the artist, song title and song's length for sessionId = 338 and itemInSession = 4

<div style="text-align: justify"> For this first query, the table to be created needs to hold the sessionId, itemInSession, artist, song, and length. The PARTITION KEY is chosen to be sessionId, while the CLUSTERING KEY is selected to be itemInSession in order to filter the entries in the table by sessionId and itemInSession.<br><br> As there is no interest in sorting the output by a particular key, the PARTITIONING and CLUSTERING KEY could also be switched. <\div>

In [21]:
## Query 1:  Return the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

query_create = '''CREATE TABLE IF NOT EXISTS artist_song_length \
                (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))
                '''

session.execute(query_create)
              

<div style="text-align: justify">The cell above creates the table artist_song_length, which includes:<\div>
    
   - sessionId as int (PARTITION KEY)
   - itemInSession as int (CLUSTERING KEY)
   - artist as text
   - song as text
   - length as float  
    
<div style="text-align: justify">The table is then used to retrieve the artist, song and length for sessionId = 338 and itemInSession = 4<\div>

In [22]:
## File path or name to dsired .csv-file ##
file = 'event_datafile_new.csv'


## Extract data from .csv-file and write it to specified table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_insert = "INSERT INTO artist_song_length (sessionId, itemInSession, artist, song, length)"
        query_insert = query_insert + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        

<div style="text-align: justify">The cell above inserts selected values into the table artist_song_length and uses the following column indices for the .csv-file:<\div>

   - 8: sessionId
   - 3: itemInSession
   - 0: artist
   - 9: song
   - 5: length

#### Do a SELECT to verify that the data have been inserted into each table

In [23]:
## SELECT statement to verify the data was entered into the table

query_select = '''SELECT artist, song, length FROM artist_song_length \
                WHERE sessionId = 338 \
                AND itemInSession = 4                
                '''
entries = session.execute(query_select)

for i in entries:
    print(i)
    

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


<div style="text-align: justify"> The cell above returns the artist, song, length for all entries with sessionId = 338 and itemInSession = 4 <\div> <br> <br>There is obviously only one entry that matches these criteria.

## Query 2: Return the name of the artist, the song and user credentials for userId = 10 and sessionId = 182

<div style="text-align: justify">For this second query, the table to be created needs to hold the userId, sessionId, itemInSession, artist, song, firstName, and lastName. Two PARTITION KEYS are used here (userId and sessionId) in order to filter the output in the query by these two keys. For the CLUSTERING KEY, itemInSession is chosen to sort the output appropriately.

In [24]:
## Query 2: Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query_create = '''CREATE TABLE IF NOT EXISTS artist_song_user \
                (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userid, sessionId), itemInSession))
                '''

session.execute(query_create)
                    

<div style="text-align: justify">The cell above creates the table artist_song_user, which includes:<\html>
    
   - userId as int (PARTITION KEY)
   - sessionId as int (PARTITION KEY)
   - itemInSession as int (CLUSTERING KEY)
   - artist as text
   - song as text
   - firstName as text
   - lastName as text
    
<div style="text-align: justify">The table is then used to retrieve the artist, song (sorted by itemInSession) and user credentials (firstName and lastName) for userId = 10 and sessionId = 182<\div>

In [25]:
## File path or name to dsired .csv-file ##
file = 'event_datafile_new.csv'


## Extract data from .csv-file and write it to specified table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_insert = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query_insert = query_insert + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

<div style="text-align: justify">The cell above inserts selected values into the table artist_song_user and uses the following column indices for the .csv-file:<\div>
    
   - 10: userId
   - 8: sessionId
   - 3: itemInSession
   - 0: artist
   - 9: song
   - 1: firstName
   - 4: lastName

In [26]:
## SELECT statement to verify the data was entered into the table

query_select = '''SELECT itemInSession, artist, song, firstName, lastName FROM artist_song_user \
                WHERE userId = 10 \
                AND sessionId = 182               
                '''
entries = session.execute(query_select)

for i in entries:
    print(i)
    

Row(iteminsession=0, artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(iteminsession=1, artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(iteminsession=2, artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(iteminsession=3, artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


<div style="text-align: justify">The cell above returns the artist, song, user firstName and lastName for all entries with userId = 10 and itemInSession = 182. The returned entries are sorted properly by itemInSession. <\div>

## Query 3: Return all user credentials (first and last name) that listened to the song 'All Hands Against His Own'

<div style="text-align: justify">For the last query, the table to be created needs to hold the song, userId, firstName, and lastName. The PARTITION KEY is set to be the song in order to filter the output in the query by it. For the CLUSTERING KEY, userId is chosen to nicely sort the output. <\div>

In [27]:
## Query 3: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query_create = '''CREATE TABLE IF NOT EXISTS song_user \
                (song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))
                '''

session.execute(query_create)
                    

<div style="text-align: justify">The cell above creates the table song_user, which includes:<\div>
    
   - song as text (PARTITION KEY)
   - userId as int (CLUSTERING KEY)
   - firstName as text
   - lastName as text
    
<div style="text-align: justify">The table is then used to retrieve all the user credentials (firstName and lastName) for song = 'All Hands Against His Own'<\div>

In [28]:
## File path or name to dsired .csv-file ##
file = 'event_datafile_new.csv'


## Extract data from .csv-file and write it to specified table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_insert = "INSERT INTO song_user (song, userId, firstName, lastName)"
        query_insert = query_insert + "VALUES (%s, %s, %s, %s)"
        
        session.execute(query_insert, (line[9], int(line[10]), line[1], line[4]))

<div style="text-align: justify">The cell above inserts selected values into song_user and uses the following column indices for the .csv-file:<\div>
    
   - 9: song
   - 10: userId
   - 1: firstName
   - 4: lastName

In [29]:
## SELECT statement to verify the data was entered into the table

query_select = '''SELECT firstName, lastName, userId FROM song_user \
                WHERE song = 'All Hands Against His Own'               
                '''
entries = session.execute(query_select)

for i in entries:
    print(i)
  

Row(firstname='Jacqueline', lastname='Lynch', userid=29)
Row(firstname='Tegan', lastname='Levine', userid=80)
Row(firstname='Sara', lastname='Johnson', userid=95)


<div style="text-align: justify">The cell above returns the user firstName and lastName for all entries with song = 'All Hands Against His Own<\div><br><br>

<div style="text-align: justify">The returned entries are sorted by userId.<\div>

### Drop the tables before closing out the sessions

In [62]:
## Drop the table before closing out the sessions

In [30]:
query_0 = 'DROP TABLE artist_song_length'
query_1 = 'DROP TABLE artist_song_user'
query_2 = 'DROP TABLE song_user'

session.execute(query_0)
session.execute(query_1)
session.execute(query_2)

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()